# Machine Learning Models Based on Recent Traffic History

In this notebook, we train and evaluate multiple machine learning models using historical traffic data only. The dataset includes engineered lag features (previous severity levels at 1h and 2h intervals) as predictors. 

We compare the performance of three models — Random Forest, Logistic Regression, and Gradient Boosting — to establish a baseline for how well traffic severity can be predicted using only recent history. I picked 3 diverse models, i.e. random forest that is tree-based, logistic regresion that is linear and gradient boosting (ensemble boosting).



In [2]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import joblib

df = pd.read_csv("../data/engineered_traffic_with_lags.csv")

# Define features and target
features = ['prev_1h_severity', 'prev_2h_severity', 'hour', 'day_of_week', 'is_weekend', 'is_rush_hour']
X = df[features]
y = df['severity_level']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define models
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42)
}


#  Train , evaluate and save each model


for name, model in models.items():
    print(f"\nModel: {name}")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred, zero_division=0))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

    # Save model
    filename = f"../models/7a_{name.lower().replace(' ', '_')}.joblib"
    joblib.dump(model, filename)
    print(f"Saved model to {filename}")




Model: Random Forest
              precision    recall  f1-score   support

           0       0.79      0.99      0.88     12622
           1       0.19      0.01      0.01      2365
           2       0.23      0.01      0.01       911

    accuracy                           0.79     15898
   macro avg       0.40      0.34      0.30     15898
weighted avg       0.67      0.79      0.70     15898

Confusion Matrix:
 [[12550    58    14]
 [ 2346    16     3]
 [  896    10     5]]
Saved model to ../models/7a_random_forest.joblib

Model: Logistic Regression
              precision    recall  f1-score   support

           0       0.79      1.00      0.89     12622
           1       0.00      0.00      0.00      2365
           2       0.00      0.00      0.00       911

    accuracy                           0.79     15898
   macro avg       0.26      0.33      0.30     15898
weighted avg       0.63      0.79      0.70     15898

Confusion Matrix:
 [[12622     0     0]
 [ 2365     0   

### Observations:

- All models reach high accuracy (79%) — but this is misleading because almost all samples are class 0.
Class imbalance dominates the results.

- Random Forest at least predicts a few minority class samples (class 1 & 2), but still it's very weak (it seems it's a bit more robust under inbalance).

- Logistic Regression and Gradient Boosting entirely collapse: they never predict anything except class 0. It is probably because, inn this notebook, the models are only using recent history features (lags, etc). These features may not carry enough signal to distinguish class 1 and 2.

- These results highlight the limitations of using only recent history and motivate the need to incorporate more informative features in subsequent modeling efforts.



### Note 

These models offer a more realistic baseline than those in Notebook 3, they show the predictive value of temporal context over structural features. However, of course, there's still a lot to be added to make them more meaningful.